# Data Preprocessing and Feature Engineering for Fraud Detection

This notebook performs data preprocessing and feature engineering on the fraud detection datasets:
1. Fraud_Data.csv - E-commerce transaction data
2. IpAddress_to_Country.csv - IP to country mapping
3. creditcard.csv - Bank transaction data

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the src directory to the path
if os.path.abspath('..') not in sys.path:
    sys.path.append(os.path.abspath('..'))

# Import custom modules
from src.load_data import load_data
from src.preprocessing import (
    handle_missing_values, clean_data, encode_categorical_features,
    scale_features, handle_class_imbalance, convert_ip_to_int
)
from src.feature_engineering import (
    add_time_features, add_transaction_features,
    merge_ip_country_data, add_amount_features
)

# Set plot style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('viridis')

# Display all columns
pd.set_option('display.max_columns', None)

## 1. Load the Datasets

In [7]:
# Load the datasets
fraud_data = load_data('../data/raw/Fraud_Data.csv')
ip_country = load_data('../data/raw/IpAddress_to_Country.csv')
creditcard = load_data('../data/raw/creditcard.csv')

## 2. Preprocess Fraud_Data.csv

In [8]:
# Clean data
fraud_data = clean_data(fraud_data)

# Handle missing values
fraud_data = handle_missing_values(fraud_data)

Number of duplicates: 0
Missing values before imputation:
Series([], dtype: int64)
Missing values after imputation:
Series([], dtype: int64)


In [ ]:
# Merge with IP-to-country data
fraud_data = merge_ip_country_data(fraud_data, ip_country)

# Check the merged data
fraud_data[['ip_address', 'country']].head()

,ip_address,country
0,7.327584e+08,Unknown
1,3.503114e+08,Unknown
2,2.621474e+09,Unknown
3,3.840542e+09,Unknown
4,4.155831e+08,Unknown


In [10]:
fraud_data.head(100)

,user_id,signup_time,purchase_time,purchase_value,device_id,source,browser,sex,age,ip_address,class,country
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,SEO,Chrome,M,39.0,7.327584e+08,0.0,Unknown
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,Ads,Chrome,F,53.0,3.503114e+08,0.0,Unknown
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,SEO,Opera,M,53.0,2.621474e+09,1.0,Unknown
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,SEO,Safari,M,41.0,3.840542e+09,0.0,Unknown
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,Ads,Safari,M,45.0,4.155831e+08,0.0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...
95,10055.0,2015-04-06 23:14:54,2015-05-07 15:21:49,47.0,KDSEIVWEOCIEM,SEO,Chrome,F,36.0,2.755626e+09,0.0,Unknown
96,244618.0,2015-06-11 12:40:52,2015-08-30 08:09:07,81.0,MSPMJUTDCXZSC,SEO,FireFox,M,20.0,3.014511e+09,0.0,Unknown
97,50280.0,2015-05-24 21:16:35,2015-06-01 19:00:06,21.0,IUZMJJBRDCGBZ,Ads,Chrome,M,24.0,4.283641e+09,0.0,Unknown
98,56338.0,2015-03-18 03:36:41,2015-07-11 08:59:24,53.0,KZZDZXHXXLQFD,SEO,Chrome,M,24.0,4.227828e+09,0.0,Unknown


In [ ]:
# Add time-based features
fraud_data = add_time_features(fraud_data)

# Add transaction features
fraud_data = add_transaction_features(fraud_data)

# Check the new features
fraud_data.head()

In [11]:
# Encode categorical features
categorical_cols = ['source', 'browser', 'sex', 'country']
fraud_data = encode_categorical_features(fraud_data, categorical_cols)

# Check the encoded features
fraud_data.head()

,user_id,signup_time,purchase_time,purchase_value,device_id,age,ip_address,class,source_Ads,source_Direct,source_SEO,browser_Chrome,browser_FireFox,browser_IE,browser_Opera,browser_Safari,sex_F,sex_M,country_Unknown
0,22058.0,2015-02-24 22:55:49,2015-04-18 02:47:11,34.0,QVPSPJUOCKZAR,39.0,7.327584e+08,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
1,333320.0,2015-06-07 20:39:50,2015-06-08 01:38:54,16.0,EOGFQPIZPYXFZ,53.0,3.503114e+08,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,1359.0,2015-01-01 18:52:44,2015-01-01 18:52:45,15.0,YSSKYOSJHPPLJ,53.0,2.621474e+09,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
3,150084.0,2015-04-28 21:13:25,2015-05-04 13:54:50,44.0,ATGTXKYKUDUQN,41.0,3.840542e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
4,221365.0,2015-07-21 07:09:52,2015-09-09 18:40:53,39.0,NAUITBZFJKHWW,45.0,4.155831e+08,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [ ]:
# Drop unnecessary columns
cols_to_drop = ['user_id', 'device_id', 'ip_address', 'signup_time', 'purchase_time']
fraud_data = fraud_data.drop(cols_to_drop, axis=1)

# Check the final dataset
fraud_data.head()

In [ ]:
# Scale numerical features
fraud_data, scaler_fraud = scale_features(fraud_data)

# Check the scaled features
fraud_data.describe()

In [12]:
# Save the preprocessed data
fraud_data.to_csv('../data/processed/fraud_data_preprocessed.csv', index=False)

## 3. Preprocess creditcard.csv

In [13]:
# Clean data
creditcard = clean_data(creditcard)

# Handle missing values
creditcard = handle_missing_values(creditcard)

Number of duplicates: 1081
Missing values before imputation:
Series([], dtype: int64)
Missing values after imputation:
Series([], dtype: int64)


In [14]:
# Add time-based features
creditcard = add_time_features(creditcard)

# Add amount-based features
creditcard = add_amount_features(creditcard)

# Check the new features
creditcard.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,time_hours,time_of_day,hour_of_day,hour_sin,hour_cos,log_amount,amount_bin
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0,0.000000,0.0,0,0.0,1.0,5.014760,8
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0,0.000000,0.0,0,0.0,1.0,1.305626,1
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0,0.000278,1.0,0,0.0,1.0,5.939276,9
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0,0.000278,1.0,0,0.0,1.0,4.824306,8
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0,0.000556,2.0,0,0.0,1.0,4.262539,7


In [15]:
# Scale numerical features
creditcard, scaler_cc = scale_features(creditcard)

# Check the scaled features
creditcard.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,time_hours,time_of_day,hour_of_day,hour_sin,hour_cos,log_amount,amount_bin
count,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,283726.000000,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05,2.837260e+05
mean,1.218105e-16,-5.659779e-17,-2.404154e-18,-1.121939e-16,1.422458e-17,3.365816e-17,4.808308e-18,-1.883254e-17,-1.141973e-17,-3.190513e-17,9.215924e-18,2.724708e-17,-3.205539e-18,-1.552683e-17,1.202077e-17,1.362354e-17,-1.903289e-18,1.262181e-17,-1.522631e-17,1.254668e-17,-5.810039e-18,-9.215924e-18,1.923323e-17,2.604500e-18,1.412441e-17,4.006924e-17,-1.207086e-17,-5.860126e-18,-1.502596e-19,-5.409347e-17,0.001667,1.218105e-16,1.186049e-16,-5.128862e-17,-1.050816e-16,-5.769970e-17,1.005738e-16,1.302250e-16
std,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,0.040796,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00,1.000002e+00
min,-1.996823e+00,-2.895933e+01,-4.415594e+01,-3.203279e+01,-4.016602e+00,-8.260323e+01,-1.964022e+01,-3.548131e+01,-6.209721e+01,-1.226160e+01,-2.284160e+01,-4.709521e+00,-1.878306e+01,-5.819090e+00,-2.017886e+01,-4.918600e+00,-1.617386e+01,-2.986682e+01,-1.134527e+01,-8.868289e+00,-7.077808e+01,-4.811386e+01,-1.508956e+01,-7.184198e+01,-4.684150e+00,-1.975208e+01,-5.403358e+00,-5.702547e+01,-4.704087e+01,-3.533268e-01,0.000000,-1.996823e+00,-2.486646e+00,-2.407217e+00,-1.161524e+00,-1.183391e+00,-1.903207e+00,-1.558191e+00
25%,-8.552128e-01,-4.732329e-01,-3.620488e-01,-5.907784e-01,-5.990519e-01,-5.022907e-01,-5.765260e-01,-4.515170e-01,-1.763908e-01,-5.866090e-01,-4.962232e-01,-7.478525e-01,-4.076546e-01,-6.514439e-01,-4.473627e-01,-6.366813e-01,-5.356809e-01,-5.745927e-01,-5.965414e-01,-5.606559e-01,-2.748846e-01,-3.148664e-01,-7.489963e-01,-2.595820e-01,-5.856219e-01,-6.086746e-01,-6.781691e-01,-1.829567e-01,-1.626863e-01,-3.309625e-01,0.000000,-8.552128e-01,-6.738937e-01,-6.933643e-01,-9.556001e-01,-9.913246e-01,-7.644127e-01,-8.640676e-01
50%,-2.131081e-01,7.426456e-03,4.134568e-02,1.182157e-01,-1.363454e-02,-4.015609e-02,-2.057379e-01,3.181559e-02,1.929719e-02,-4.655399e-02,-8.527996e-02,-3.191039e-02,1.405355e-01,-1.359251e-02,5.246341e-02,5.274504e-02,7.549218e-02,-7.838176e-02,-4.367121e-03,4.464530e-03,-8.122341e-02,-4.015703e-02,9.233221e-03,-1.820863e-02,6.737087e-02,3.167716e-02,-1.085382e-01,-7.187408e-04,3.274220e-02,-2.654671e-01,0.000000,-2.131081e-01,8.069497e-02,1.635622e-01,-3.930054e-01,-1.208329e-01,-1.102319e-02,-1.699443e-01
75%,9.369423e-01,6.725541e-01,4.885028e-01,6.796321e-01,5.251188e-01,4.432736e-01,2.987636e-01,4.632165e-01,2.769666e-01,5.454844e-01,4.227584e-01,7.257918e-01,6.209995e-01,6.656182e-01,5.167783e-01,7.094403e-01,5.978627e-01,4.733517e-01,5.976293e-01,5.640338e-01,1.727570e-01,2.577181e-01,7.290881e-01,2.365720e-01,7.257359e-01,6.732285e-01,4.981036e-01,2.260181e-01,2.369592e-01,-4.378088e-02,0.000000,9.369423e-01,8.196987e-01,8.491034e-01,7.733279e-01,9.670110e-01,7.298789e-01,8.712406e-01
max,1.642362e+00,1.257179e+00,1.339762e+01,6.217985e+00,1.193504e+01,2.527211e+01,5.503507e+01,9.822550e+01,1.696961e+01,1.423708e+01,2.206100e+01,1.179787e+01,7.891145e+00,7.159011e+00,1.105478e+01,9.702455e+00,1.981693e+

In [16]:
# Save the preprocessed data
creditcard.to_csv('../data/processed/creditcard_preprocessed.csv', index=False)

## 4. Prepare Data for Modeling

In [17]:
# Prepare Fraud_Data for modeling
X_fraud = fraud_data.drop('class', axis=1)
y_fraud = fraud_data['class']

# Prepare creditcard for modeling
X_cc = creditcard.drop('Class', axis=1)
y_cc = creditcard['Class']

# Print shapes
print(f"Fraud_Data: X shape = {X_fraud.shape}, y shape = {y_fraud.shape}")
print(f"Creditcard: X shape = {X_cc.shape}, y shape = {y_cc.shape}")

Fraud_Data: X shape = (151112, 18), y shape = (151112,)
Creditcard: X shape = (283726, 37), y shape = (283726,)


In [18]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

# Fraud_Data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

# Creditcard
X_train_cc, X_test_cc, y_train_cc, y_test_cc = train_test_split(
    X_cc, y_cc, test_size=0.2, random_state=42, stratify=y_cc
)

# Print shapes
print(f"Fraud_Data: X_train shape = {X_train_fraud.shape}, X_test shape = {X_test_fraud.shape}")
print(f"Creditcard: X_train shape = {X_train_cc.shape}, X_test shape = {X_test_cc.shape}")

Fraud_Data: X_train shape = (120889, 18), X_test shape = (30223, 18)
Creditcard: X_train shape = (226980, 37), X_test shape = (56746, 37)


In [ ]:
# Handle class imbalance for Fraud_Data
X_train_fraud_resampled, y_train_fraud_resampled = handle_class_imbalance(
    X_train_fraud, y_train_fraud, method='smote', sampling_strategy=0.1
)

# Handle class imbalance for Creditcard
X_train_cc_resampled, y_train_cc_resampled = handle_class_imbalance(
    X_train_cc, y_train_cc, method='smote', sampling_strategy=0.1
)

In [ ]:
# Save the train-test split data
import joblib

# Create directory if it doesn't exist
os.makedirs('../data/processed', exist_ok=True)

# Save Fraud_Data splits
joblib.dump(X_train_fraud, '../data/processed/X_train_fraud.pkl')
joblib.dump(X_test_fraud, '../data/processed/X_test_fraud.pkl')
joblib.dump(y_train_fraud, '../data/processed/y_train_fraud.pkl')
joblib.dump(y_test_fraud, '../data/processed/y_test_fraud.pkl')
joblib.dump(X_train_fraud_resampled, '../data/processed/X_train_fraud_resampled.pkl')
joblib.dump(y_train_fraud_resampled, '../data/processed/y_train_fraud_resampled.pkl')

# Save Creditcard splits
joblib.dump(X_train_cc, '../data/processed/X_train_cc.pkl')
joblib.dump(X_test_cc, '../data/processed/X_test_cc.pkl')
joblib.dump(y_train_cc, '../data/processed/y_train_cc.pkl')
joblib.dump(y_test_cc, '../data/processed/y_test_cc.pkl')
joblib.dump(X_train_cc_resampled, '../data/processed/X_train_cc_resampled.pkl')
joblib.dump(y_train_cc_resampled, '../data/processed/y_train_cc_resampled.pkl')

# Save scalers
joblib.dump(scaler_fraud, '../data/processed/scaler_fraud.pkl')
joblib.dump(scaler_cc, '../data/processed/scaler_cc.pkl')

print("Data splits saved successfully.")

## 5. Summary of Preprocessing Steps

### Fraud_Data.csv
1. Cleaned data by removing duplicates and correcting data types
2. Handled missing values using imputation
3. Merged with IP-to-country data for geolocation analysis
4. Added time-based features (hour_of_day, day_of_week, time_since_signup)
5. Added transaction features (user_transaction_count, time_since_last_transaction, etc.)
6. Encoded categorical features (source, browser, sex, country)
7. Dropped unnecessary columns (user_id, device_id, ip_address, signup_time, purchase_time)
8. Scaled numerical features
9. Split data into training and testing sets
10. Handled class imbalance using SMOTE

### creditcard.csv
1. Cleaned data by removing duplicates and correcting data types
2. Handled missing values using imputation
3. Added time-based features
4. Added amount-based features
5. Scaled numerical features
6. Split data into training and testing sets
7. Handled class imbalance using SMOTE